In [1]:
def show_usage():
    print("usage: python restore-nfs-mount.py -f <vip file> -v <view> -m <mount>")
    print(" -f : file that has vip list")
    print(" -v : Cohesity view")
    print(" -m : mount-prefix (like /coh/ora)")

In [2]:
import re
import os
import sys
import argparse
import subprocess
from subprocess import call

In [4]:
argv = sys.argv[1:]
argv = [i.lower() for i in argv]
if '--help' in argv or 'help' in argv or '-help' in argv:
    show_usage()
    print("exit")
    exit(0)
parser = argparse.ArgumentParser(description = "Parser for nfs-mount", conflict_handler='resolve')

parser.add_argument("-f", "--vipfile", help = "file that has vip list", 
                    required = True, default = "")
parser.add_argument("-v", "--view", help = "Cohesity view", 
                    required = True, default = "")
parser.add_argument("-m", "--mount", help = "mount-prefix (like /mnt/ora)",
                    required = True, default = "")
args = parser.parse_args(argv)
if (os.path.exists(args.vipfile)):
    print("file {} provided exist, script continue".format(args.vipfile))
else:
    print("file {} provided does not exist".format(args.vipfile))
i = 1
vipfile = open(args.vipfile, "r")
Lines = vipfile.readlines() 
for line in Lines:
    if (os.path.isdir("{0}{1}".format(mount, i)) == False):
        print("Directory {0}{1} does not exist, create it".format(mount, i))
        try:
            os.system("mkdir -p {0}{1}".format(mount, i))
        except OSError:
            print("Exception created with create new directory {0}{1}".format(mount, i))
            exit(1)
    temp = ""
    line = line.strip()
    print("Check whether IP {} can be connected".format(line))
    
    if (line != "" and len(line) != 0):
        temp = "/bin/ping {} -c 2".format(line)
        cmd = ["/bin/ping", "{}".format(line), "-c", "2"]
        proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        o, e = proc.communicate()
        output = o.decode('ascii')
        if(len(e.decode('ascii'))== 0):
            output = output.splitlines()
        else:
            print(e)
            print("Error when running ps -ef")
            exit(2)
        result = []
        for x in output:
            if re.search("error", x):
                result.append(x)
        if (len(result)!=0):
            print("error: IP {} can't be connected. It may not be a valid IP. Skip this IP".format(line))
        else:
            print("IP {} can be connected".format(line))
            
            if os.path.ismount("{0}{1}".format(mount, i)):
                print("{0}{1} is mount point, will unmount it".format(mount, i))
                os.unmount("{0}{1}".format(mount, i))
                os.mount() ##TODO

usage: ipykernel_launcher.py [-h] -f VIPFILE -v VIEW -m MOUNT
ipykernel_launcher.py: error: the following arguments are required: -v/--view, -m/--mount


SystemExit: 2